In [1]:
import sys
import json
import logging
from structure_user_input import structure_user_input, convert_to_dict
from retrieval_utils import width_cabinet,calculate_rear_panels_constrained, construct_file_path
from component_retriever import retrieve_modules, retriever, index
from dotenv import load_dotenv, find_dotenv
import os

Dotenv path: /Users/nikolaoskaliorakis/Documents/Lista/lista_generator_mike/lista_generator_mike/LLM_chain/LLM_chain/.env
OpenAI API Key: sk-proj-AS...

Searches saved to: /Users/nikolaoskaliorakis/Documents/Lista/lista_generator_mike/lista_generator_mike/LLM_chain/LLM_chain/searches/search_20241204_102636.json
../../assets/components/cabinet_with_hinged_doors_1.usda
INFO:llama_index.core.indices.loading:Loading all indices.


In [2]:
# Set up logging
logging.basicConfig(level=logging.INFO, format='%(asctime)s - %(levelname)s - %(message)s')

# Load .env
dotenv_path = find_dotenv()
logging.info(f"Dotenv path: {dotenv_path}")
load_dotenv(dotenv_path)

OPENAI_API_KEY = os.getenv("OPENAI_API_KEY")
logging.info(f"OpenAI API Key: {OPENAI_API_KEY[:10]}...") # Only log the first 10 characters for security

INFO:root:Dotenv path: /Users/nikolaoskaliorakis/Documents/Lista/lista_generator_mike/lista_generator_mike/LLM_chain/LLM_chain/.env
INFO:root:OpenAI API Key: sk-proj-AS...


In [3]:

user_input = """I need a modular workspace setup that includes three cabinets: 
    One bin cabinet on the left, small-sized with a compact footprint, 
    one rolling cabinet in the center, small-sized with smooth castors for mobility, designed with multiple drawers, 
    and one drawer cabinet on the right, small-sized and optimized for tool storage. 
    The workbench should have an ABS top for durability and easy maintenance. 
    The rear of the setup requires three rows of rear panels, designed for modular functionality and support."""
    #input("\nEnter your requirements: ")

In [4]:
# Step 1: Structure user input
logging.info("Step 1: Structuring user input")
structured_input = structure_user_input(user_input)
if not structured_input:
    raise ValueError("No structured input received")
searches = convert_to_dict(structured_input)
logging.info(f"Structured input: {(json.dumps(searches, indent=2))}")


def print_readable_components(searches):
    print("Components:")
    for comp in searches['components']:
        print(f"\n• {comp['category']}:")
        for key, value in comp.items():
            if key != 'category':
                print(f"  - {key}: {value}")

print_readable_components(searches)

INFO:root:Step 1: Structuring user input
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/chat/completions "HTTP/1.1 200 OK"
INFO:root:Structured input: {
  "metadata": {
    "createdAt": "2024-12-04T10:26:46.794402",
    "timestamp": "20241204_102646"
  },
  "components": [
    {
      "category": "Cabinet",
      "requirements": [
        "bin",
        "left",
        "compact footprint"
      ],
      "size": "Small"
    },
    {
      "category": "Cabinet",
      "requirements": [
        "rolling",
        "center",
        "smooth castors",
        "multiple drawers"
      ],
      "size": "Small"
    },
    {
      "category": "Cabinet",
      "requirements": [
        "drawer",
        "right",
        "optimized for tool storage"
      ],
      "size": "Small"
    },
    {
      "category": "Workbench Top",
      "requirements": [
        "ABS",
        "durable",
        "easy maintenance"
      ],
      "size": null
    },
    {
      "category": "Rear Panels",
     

In [5]:
# Step 2: Retrieve components for each part of the structured input
cabinets = [c for c in searches['components'] if c['category'] == "Cabinet"]
for cabinet in cabinets:
    try:
        retrieved_modules = retrieve_modules(str(cabinet))
        if retrieved_modules:
            # Assuming you want the first result's default_prim
            cabinet['filepath'] = retrieved_modules[0]['default_prim']
        else:
            cabinet['filepath'] = None
    except Exception as e:
        logging.error(f"Failed to retrieve module for cabinet: {cabinet}. Error: {e}")
        cabinet['filepath'] = None


workbenches = [c for c in searches['components'] if c['category'] == "Workbench Top"]
for workbench in workbenches:
    try:
        retrieved_modules = retrieve_modules(str(workbench))
        if retrieved_modules:
            # Assuming you want the first result's default_prim
            workbench['filepath'] = retrieved_modules[0]['default_prim']
        else:
            workbench['filepath'] = None
    except Exception as e:
        logging.error(f"Failed to retrieve module for workbench: {workbench}. Error: {e}")
        workbench['filepath'] = None
    

INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.openai.com/v1/embeddings "HTTP/1.1 200 OK"


In [6]:
# Step 3: Add filepaths to the json
assembly_data_model = searches  # This is a dictionary

# Overwrite the cabinets and workbench with the updated data
# Remove existing cabinets and workbench top from the components
assembly_data_model['components'] = [
    component for component in assembly_data_model['components']
    if component['category'] not in ["Cabinet", "Workbench Top"]
]
# Add the updated cabinets and workbench to the components
assembly_data_model['components'].extend(cabinets)
assembly_data_model['components'].extend(workbenches)
logging.info("Assembly data model after adding cabinets and workbenches: %s", json.dumps(assembly_data_model, indent=2))

print_readable_components(assembly_data_model)

INFO:root:Assembly data model after adding cabinets and workbenches: {
  "metadata": {
    "createdAt": "2024-12-04T10:26:46.794402",
    "timestamp": "20241204_102646"
  },
  "components": [
    {
      "category": "Rear Panels",
      "requirements": [
        "modular functionality",
        "support"
      ],
      "size": null
    },
    {
      "category": "Cabinet",
      "requirements": [
        "bin",
        "left",
        "compact footprint"
      ],
      "size": "Small",
      "filepath": "hinged_door_cabinet_with_bin_1"
    },
    {
      "category": "Cabinet",
      "requirements": [
        "rolling",
        "center",
        "smooth castors",
        "multiple drawers"
      ],
      "size": "Small",
      "filepath": "rolling_cabinet_7"
    },
    {
      "category": "Cabinet",
      "requirements": [
        "drawer",
        "right",
        "optimized for tool storage"
      ],
      "size": "Small",
      "filepath": "drawer_cabinet_8"
    },
    {
      "categ

In [9]:
# Step 4: Calculate length and config of rear panels
# Calculate total width of cabinets using the width_cabinet function
cabinet_filepaths = [construct_file_path(cabinet['filepath']) for cabinet in cabinets if cabinet['filepath']]
cabinet_widths = width_cabinet(cabinet_filepaths)
total_cabinet_width = round(sum(cabinet_widths) * 1000)
logging.info(f"Total width of cabinets: {total_cabinet_width}")

# Calculate rear panel configuration
panels_config = calculate_rear_panels_constrained(total_cabinet_width)
logging.info(f"Rear panel configuration: {panels_config }")

# Step 5: Add rear panels to the assembly data model

# Check if there is a category "Rear Panels" in the assembly data model
rear_panels_components = [
    component for component in assembly_data_model['components']
    if component['category'] == "Rear Panels"
]

if not rear_panels_components:
    logging.info("No 'Rear Panels' category found in the assembly data model.")
else:
    logging.info("Adding rear panels to the assembly data model")
    rear_panels = []

    # L_panels['quantity_a'] and L_panels['quantity_b'] are calculated in step 4
    quantity_a = panels_config['a']
    quantity_b = panels_config['b']

    # Nest rear panels as children under 'size' in the existing rear panel component
    for panel in rear_panels_components:
        panel['size'] = []
        if quantity_a > 0:
            panel['size'].append({
                'filepath': 'rear_panel_with_keyholes_4',
                'quantity': quantity_a
            })
            logging.info(f"Adding {quantity_a} rear panels to the nested panels under 'size'")

        if quantity_b > 0:
            panel['size'].append({
                'filepath': 'rear_panel_with_keyholes_5',
                'quantity': quantity_b
            })
            logging.info(f"Adding {quantity_b} rear panels to the nested panels under 'size'")

    logging.info("Assembly data model after nesting rear panels under 'size': %s", json.dumps(assembly_data_model, indent=2))

INFO:root:Total width of cabinets: 1998
INFO:root:Rear panel configuration: {'a': 0, 'b': 2, 'spacing': 2}
INFO:root:Adding rear panels to the assembly data model
INFO:root:Adding 2 rear panels to the nested panels under 'size'
INFO:root:Assembly data model after nesting rear panels under 'size': {
  "metadata": {
    "createdAt": "2024-12-04T10:26:46.794402",
    "timestamp": "20241204_102646",
    "W_tot_cabinets": 1998,
    "spacing": 2,
    "number_of_cabinets": 3
  },
  "components": [
    {
      "category": "Rear Panels",
      "requirements": [
        "modular functionality",
        "support"
      ],
      "size": [
        {
          "filepath": "rear_panel_with_keyholes_5",
          "quantity": 2
        }
      ]
    },
    {
      "category": "Cabinet",
      "requirements": [
        "bin",
        "left",
        "compact footprint"
      ],
      "size": "Small",
      "filepath": "hinged_door_cabinet_with_bin_1"
    },
    {
      "category": "Cabinet",
      "requ

In [10]:
 #Step 6: append metadata (don't overwrite)
if 'metadata' not in assembly_data_model:
    assembly_data_model['metadata'] = {}

assembly_data_model['metadata'].update({
    'W_tot_cabinets': total_cabinet_width,
    'spacing': panels_config['spacing'],
    'number_of_cabinets': len(cabinets)
})
logging.info(f"Metadata appended to assembly data model: {json.dumps(assembly_data_model, indent=2)}")

INFO:root:Metadata appended to assembly data model: {
  "metadata": {
    "createdAt": "2024-12-04T10:26:46.794402",
    "timestamp": "20241204_102646",
    "W_tot_cabinets": 1998,
    "spacing": 2,
    "number_of_cabinets": 3
  },
  "components": [
    {
      "category": "Rear Panels",
      "requirements": [
        "modular functionality",
        "support"
      ],
      "size": [
        {
          "filepath": "rear_panel_with_keyholes_5",
          "quantity": 2
        }
      ]
    },
    {
      "category": "Cabinet",
      "requirements": [
        "bin",
        "left",
        "compact footprint"
      ],
      "size": "Small",
      "filepath": "hinged_door_cabinet_with_bin_1"
    },
    {
      "category": "Cabinet",
      "requirements": [
        "rolling",
        "center",
        "smooth castors",
        "multiple drawers"
      ],
      "size": "Small",
      "filepath": "rolling_cabinet_7"
    },
    {
      "category": "Cabinet",
      "requirements": [
       